In [43]:
import pandas as pd
import numpy as np

In [44]:
df_listino = pd.read_csv('data_demo/art_costi.csv')
df_ddt = pd.read_csv('data_demo/ddt_righe.csv')

df_ddt = df_ddt[df_ddt["Data_documento"] > "2024-03-10"]
# remove from listino al the products that are not in the ddt
df_listino = df_listino[df_listino["Codice_articolo"].isin(df_ddt["Codice_articolo"])]

In [45]:
df_listino.head()

,Codice_articolo,Codice_variante,Descrizione,Costo,Data_riferimento,Categoria
26,017.PM152,1,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
27,017.PM152,2,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
28,017.PM152,3,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
29,017.PM152,4,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
30,017.PM152,5,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri


In [46]:
df_ddt#.head()

,COD_CF_Fornitore,Data_documento,ID_documento,Descrizione,Codice_articolo,Codice_variante,Codice_categoria,Unita_misura,Quantita_um,Prezzo_netto_vu,importo
20369,1964,2024-03-12,2024-BADDV-0000404,VETROCAMERA OBLO' PER PORTA TRASPARENTE S45,045.OB.P1T,0,20.10.066,PZ,4.00,0.00000,0.00
20370,1964,2024-03-11,2024-BADDV-0000405,VETROCAMERA OBLO' PER PORTA TRASPARENTE S45,045.OB.P1T,0,20.10.066,PZ,12.00,0.00000,0.00
20408,1571,2024-03-11,2024-BADDV-0000429,NASTRO ACCIAIO ZINCATO_x000D_DX51D Z100_x000D_...,017.PM001,29,10.30.10.10.30.05,KG,570.00,0.90000,513.00
20409,1571,2024-03-11,2024-BADDV-0000429,NASTRO ACCIAIO ZINCATO,017.PM001,29,10.30.10.10.30.05,KG,12000.00,0.92000,11040.00
20410,597,2024-03-12,2024-BADDV-0000435,CARPENTERIA METALLICA,017.PM507,0,10.30.50.10.190,NR,1.00,8500.00000,8500.00
20411,5624,2024-03-13,2024-BADDV-0000436,PROFILI DOORMATICA SCORREVOLE AUT.,017.PM507,0,10.30.50.10.190,NR,22.00,0.00000,0.00
20412,1964,2024-03-15,2024-BADDV-0000439,VETRO STRATIFICATO 33.1,017.PM152,277,10.30.40.10.10,MQ,7.97,20.00000,159.40
20413,1964,2024-03-15,2024-BADDV-0000439,VETRO STRATIFICATO 33.1,017.PM152,300,10.30.40.10.10,MQ,0.98,20.00000,19.60
20414,1964,2024-03-15,2024-BADDV-0000439,VETRO STRATIFICATO 33.1,017.PM152,301,10.30.40.10.10,MQ,2.40,20.00000,48.00
20415,1964,2024-03-15,2024-BADDV-0000439,VETRO TRASPARENTE STRATIFICATO 3+3 F.G. ANTA I...,090.VST33,224,10.30.40.10.50,MQ,4.56,25.00000,114.00


In [47]:
dict_entities = {}
ids_products_list = []
ids_clienti_list = []
ids_azien_list = []

In [48]:
# prodotti in listino
for index in range(len(df_listino)):
    cod_articolo = df_listino.iloc[index]["Codice_articolo"]
    ids_products_list.append(cod_articolo)
    variante = int(df_listino.iloc[index]["Codice_variante"])
    costo = df_listino.iloc[index]["Costo"]
    data_riferimento = df_listino.iloc[index]["Data_riferimento"]
    categoria = df_listino.iloc[index]["Categoria"]
    descrizione = df_listino.iloc[index]["Descrizione"]
    dict_entities["prod_"+cod_articolo] = {
        "cod_articolo": cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [49]:
# prodotti inventati
for index in range(5):
    cod_articolo = "prod_"+str(index)
    ids_products_list.append(cod_articolo)
    variante = int(np.random.choice([0, 1, 2]))
    costo = np.random.uniform(10000, 20000)
    data_riferimento = "2024-03-17"
    categoria = "level2"
    descrizione = "prodotto inventato"
    dict_entities["prod_"+cod_articolo] = {
        "cod_articolo": cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [50]:
#fornitori e ddt
for index in range(len(df_ddt)):
    cod_fornitore = int(df_ddt.iloc[index]["COD_CF_Fornitore"])
    dict_entities["fornitore_" + str(cod_fornitore)] = {
        "cod_fornitore": cod_fornitore
    }
    data_documento = df_ddt.iloc[index]["Data_documento"]
    id_documento = df_ddt.iloc[index]["ID_documento"]
    descrizione = df_ddt.iloc[index]["Descrizione"]
    codice_articolo = df_ddt.iloc[index]["Codice_articolo"]
    codice_variante = int(df_ddt.iloc[index]["Codice_variante"])
    codice_categoria = df_ddt.iloc[index]["Codice_categoria"]
    unita_misura = df_ddt.iloc[index]["Unita_misura"]
    quantita_um = df_ddt.iloc[index]["Quantita_um"]
    prezzo_netto_vu = df_ddt.iloc[index]["Prezzo_netto_vu"]
    importo = df_ddt.iloc[index]["importo"]

    if id_documento not in dict_entities:
        dict_entities["documento_" + id_documento] = {
            "data_documento": data_documento,
            "id_documento": id_documento,
            "venduto": [],
            "importo": importo,
            "cod_fornitore": cod_fornitore,
            "azienda": "mangini_" + str(np.random.choice([1, 2, 3]))
        }
    dict_entities["documento_" + id_documento]["venduto"].append(
        {
        "codice_articolo": codice_articolo,
        "codice_variante": codice_variante,
        "descrizione": descrizione,
        "codice_categoria": codice_categoria,
        "unita_misura": unita_misura,
        "quantita_um": quantita_um,
        "prezzo_netto_vu": prezzo_netto_vu,
        }
    )

In [51]:
#clienti
for i in range(10):
    dict_entities["cliente_"+str(i)] = { "id_cliente": "cliente_"+str(i)}
    ids_clienti_list.append("cliente_"+str(i))

In [52]:
# aziende
for i in range(2):
    dict_entities["mangini_"+str(i)] = { "id_azienda": "mangini_"+str(i+1)}
    ids_azien_list.append("mangini_"+str(i))

In [53]:
# preventivi
for i in range(10):
    data_documento =  "2024-03-17"
    id_documento = "preventivo_"+str(i)
    descrizione = "preventivo inventato"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities["prod_"+codice_articolo]
        codice_variante = int(product["variante"])
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": int(codice_variante),
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "da_vendere": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda,
        "status": np.random.choice(["in attesa", "accettato"])
        }

In [54]:
# fatture
for i in range(20):
    data_documento =  "2024-03-17"
    id_documento = "fattura_"+str(i)
    descrizione = "fattura inventata"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities["prod_"+codice_articolo]
        codice_variante = int(product["variante"])
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": int(codice_variante),
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "venduto": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda
        }

In [55]:
dict_entities

{'prod_017.PM152': {'cod_articolo': '017.PM152',
  'variante': 0,
  'costo': 10000.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO 33.1'},
 'prod_090.VST33': {'cod_articolo': '090.VST33',
  'variante': 0,
  'costo': 24.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE STRATIFICATO 3+3 F.G. ANTA INTELAIATA'},
 'prod_090.VST55': {'cod_articolo': '090.VST55',
  'variante': 0,
  'costo': 37.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO TRASPARENTE 5+5'},
 'prod_045.OB.P1T': {'cod_articolo': '045.OB.P1T',
  'variante': 0,
  'costo': 75.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Visive',
  'descrizione': "VETROCAMERA OBLO' PER PORTA 400X700 R100 TRASPARENTE S45"},
 'prod_052.MP.VT.POR.TR.45.04': {'cod_articolo': '052.MP.VT.POR.TR.45.04',
  'variante': 0,
  'costo': 112.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Visi

In [56]:
keys = dict_entities.keys()
keys = sorted(keys)
keys

['cliente_0',
 'cliente_1',
 'cliente_2',
 'cliente_3',
 'cliente_4',
 'cliente_5',
 'cliente_6',
 'cliente_7',
 'cliente_8',
 'cliente_9',
 'documento_2024-BADDV-0000404',
 'documento_2024-BADDV-0000405',
 'documento_2024-BADDV-0000429',
 'documento_2024-BADDV-0000435',
 'documento_2024-BADDV-0000436',
 'documento_2024-BADDV-0000439',
 'documento_2024-BADDV-0000440',
 'documento_2024-BADDV-0000445',
 'documento_2024-BADDV-0000447',
 'documento_2024-BADDV-0000448',
 'fattura_0',
 'fattura_1',
 'fattura_10',
 'fattura_11',
 'fattura_12',
 'fattura_13',
 'fattura_14',
 'fattura_15',
 'fattura_16',
 'fattura_17',
 'fattura_18',
 'fattura_19',
 'fattura_2',
 'fattura_3',
 'fattura_4',
 'fattura_5',
 'fattura_6',
 'fattura_7',
 'fattura_8',
 'fattura_9',
 'fornitore_1571',
 'fornitore_1964',
 'fornitore_3493',
 'fornitore_5624',
 'fornitore_584',
 'fornitore_597',
 'mangini_0',
 'mangini_1',
 'preventivo_0',
 'preventivo_1',
 'preventivo_2',
 'preventivo_3',
 'preventivo_4',
 'preventivo_5'

In [57]:
import json
with open("dict_entities.json", "w") as fp:
    json.dump(dict_entities , fp) 

In [58]:
entities_types = ["prod", "fornitore", "documento", "cliente", "mangini", "preventivo", "fattura"]
dict_types_list = {"prod":[], "fornitore":[], "documento":[], "cliente":[], "mangini":[], "preventivo":[], "fattura":[]}
for key in dict_entities:
    key_first_part = key.split("_")[0]
    dict_types_list[key_first_part].append(dict_entities[key])

In [59]:
for key in entities_types:
    df = pd.DataFrame(dict_types_list[key])
    df.to_csv("data_demo/df_"+key+".csv", index=False)

In [60]:
df_prod = pd.DataFrame(dict_types_list["fattura"])
df_prod

,data_documento,id_documento,venduto,importo,id_cliente,id_azienda
0,2024-03-17,fattura_0,"[{'codice_articolo': '017.PM152', 'codice_vari...",30000.610208,cliente_0,mangini_1
1,2024-03-17,fattura_1,"[{'codice_articolo': '090.VST33', 'codice_vari...",10024.954253,cliente_6,mangini_1
2,2024-03-17,fattura_2,"[{'codice_articolo': '017.PM152', 'codice_vari...",9999.925004,cliente_8,mangini_0
3,2024-03-17,fattura_3,"[{'codice_articolo': '090.VST33', 'codice_vari...",10025.212778,cliente_5,mangini_0
4,2024-03-17,fattura_4,"[{'codice_articolo': '017.PM152', 'codice_vari...",10001.594577,cliente_6,mangini_1
5,2024-03-17,fattura_5,"[{'codice_articolo': '017.PM152', 'codice_vari...",10000.305945,cliente_1,mangini_1
6,2024-03-17,fattura_6,"[{'codice_articolo': '017.PM507', 'codice_vari...",30000.621370,cliente_7,mangini_1
7,2024-03-17,fattura_7,"[{'codice_articolo': '017.PM152', 'codice_vari...",20000.538865,cliente_8,mangini_1
8,2024-03-17,fattura_8,"[{'codice_articolo': '017.PM152', 'codice_vari...",19999.893463,cliente_6,mangini_0
9,2024-03-17,fattura_9,"[{'codice_articolo': '017.PM152', 'codice_vari...",10000.278615,cliente_6,mangini_0
